In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dfg = pd.read_csv('testdata/00testestes.csv')
mnb = pd.read_stata('testdata/GLAD.dta')
df = pd.read_csv('testdata/0.OAI18Feb.csv').drop(columns='Unnamed: 0')
oai = pd.read_csv("testdata/00testestesoai13feb.csv").drop(columns='Unnamed: 0')

## GLAD

##### Hip patients are excluded:

In [3]:
def knee_pain(mnb):
    df_knee_pain = mnb[(mnb['ptb_3754'] == 5.0) | (mnb['ptb_3754'] == 6.0)];
    return df_knee_pain

mnb = knee_pain(mnb)
print(mnb.shape)

(40411, 1240)


In [4]:
mnb.pt12_39588_2.value_counts(dropna=False)

pt12_39588_2
NaN    38939
0.0     1308
1.0      164
Name: count, dtype: int64

In [6]:
variables = ["fysb_3642", "fys3_9699", "fysb_3646", "pt12_35377", "pt12_16340", "pt12_35374",
             "fysb_op", "fysb_19334", "fysb_19335", 'ptb_10291_3', 'ptb_10291_4', 'pt12_39588_2',
             "ptb_11836", #joint replacement in hip or knee
             'pt12_10295' #knee or hip replacement since starting the GLAD
             ]
mnb = mnb[variables]
print(mnb.shape)

(40411, 14)


In [7]:
# Reset the index and rename the index column if it conflicts
mnb = mnb.reset_index().rename(columns={'index': 'mnb_index'})

# Merge the two dataframes, keeping all rows from dfg
merged_df = pd.merge(dfg, mnb, how='left', left_on='Unnamed: 0', right_on='mnb_index')

# Drop the 'mnb_index' column if it's no longer needed
merged_df = merged_df.drop(columns=['mnb_index'])

# Check the shape and preview the resulting dataframe
print(merged_df.shape)

(7603, 29)


In [8]:
# Reset the index and drop it as a column
mnb = mnb.reset_index(drop=True)

# Merge the two dataframes
merged_df = pd.merge(dfg, mnb, how='left', left_on='Unnamed: 0', right_index=True)

# Check the shape and preview the resulting dataframe
print(merged_df.shape)



(7603, 30)


In [9]:
merged_df.isna().sum()

Unnamed: 0                       0
Age                              0
Gender                           0
BMI                              0
Depression                       0
Employment_status                0
Baseline_Physical_Activity       0
Followup_Physical_Activity       0
Baseline_Pain                    0
Followup_Pain                    0
Baseline_Quality_of_life         0
Followup_Quality_of_life         0
PA_change                        0
QOL_change                       0
pain_change                      0
mnb_index                        0
fysb_3642                        0
fys3_9699                     2213
fysb_3646                        0
pt12_35377                    7603
pt12_16340                    2971
pt12_35374                    7603
fysb_op                          0
fysb_19334                    7603
fysb_19335                    7603
ptb_10291_3                     52
ptb_10291_4                     52
pt12_39588_2                  7603
ptb_11836           

In [10]:
glad_variables = ["fysb_3642", "fys3_9699", "fysb_3646", "pt12_35377", "pt12_16340", "pt12_35374",
             "fysb_op", "fysb_19334", "fysb_19335",
             'ptb_11836', 'ptb_10291_3', 'ptb_10291_4', 'pt12_10295', 'pt12_39588_2'
             ]

# Display value counts including NaN for each variable
for var in glad_variables:
    print(f"\nValue counts for {var}:\n")
    print(merged_df[var].value_counts(dropna=False))


Value counts for fysb_3642:

fysb_3642
2.0    4882
1.0    2721
Name: count, dtype: int64

Value counts for fys3_9699:

fys3_9699
2.0    5102
NaN    2213
1.0     288
Name: count, dtype: int64

Value counts for fysb_3646:

fysb_3646
1.0    5646
2.0    1957
Name: count, dtype: int64

Value counts for pt12_35377:

pt12_35377
NaN    7603
Name: count, dtype: int64

Value counts for pt12_16340:

pt12_16340
NaN    2971
2.0    2553
1.0    2079
Name: count, dtype: int64

Value counts for pt12_35374:

pt12_35374
NaN    7603
Name: count, dtype: int64

Value counts for fysb_op:

fysb_op
2    5189
1    2414
Name: count, dtype: int64

Value counts for fysb_19334:

fysb_19334
NaN    7603
Name: count, dtype: int64

Value counts for fysb_19335:

fysb_19335
NaN    7603
Name: count, dtype: int64

Value counts for ptb_11836:

ptb_11836
2.0    6789
1.0     762
NaN      52
Name: count, dtype: int64

Value counts for ptb_10291_3:

ptb_10291_3
0.0    7315
1.0     236
NaN      52
Name: count, dtype: int64

Val

In [11]:
merged_df.shape

(7603, 30)

##### How many percent of patients had surgery befor and after GLAD? 5.3.2025

In [ ]:
total_patients = len(merged_df)

# Patients who had a knee or hip replacement before GLA:D®
prior_replacement = merged_df[merged_df['ptb_11836'] == 1]

# Patients who had a knee or hip replacement since starting GLA:D®
replacement_since_GLAD = merged_df[merged_df['pt12_10295'] == 1]

# Patients who had both replacements (one before and one after GLA:D®)
both_replacements = merged_df[(merged_df['ptb_11836'] == 1) & (merged_df['pt12_10295'] == 1)]


percent_prior_replacement = (len(prior_replacement) / total_patients) * 100
percent_both_replacements = (len(both_replacements) / total_patients) * 100

print(f"Percentage of patients who had a joint replacement before GLA:D®: {percent_prior_replacement:.2f}%")
print(f"Percentage of patients who had both replacements (before and since GLA:D®): {percent_both_replacements:.2f}%")



Percentage of patients who had a joint replacement before GLA:D®: 10.02%
Percentage of patients who had both replacements (before and since GLA:D®): 1.00%


## OAI

In [21]:
print("shape df: ", df.shape)
print("shape oai: ", oai.shape)

shape df:  (4796, 40)
shape oai:  (1156, 15)


In [22]:
df

,ID,P02SEX,V00COHORT,V00AGE,P01BMI,V00CESD6,V00P7LKRCV,V00P7RKRCV,V00KOOSQOL,V00CEMPLOY,...,V01KRSR12,V01ARTR12,V01KSRGL12,V01KRSL12,V01EKRSL,V01EKRSR,V10CHIRCUR,V10HOMECV,V10NSAIDS,V10NSAIDRX
0,b'9000099',1.0,1.0,59.0,23.8,1.0,0.0,4.0,25.00,1.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0
1,b'9000296',1.0,2.0,69.0,29.8,1.0,0.0,0.0,100.00,1.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,b'9000622',2.0,1.0,71.0,22.7,1.0,0.0,3.0,50.00,4.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b'9000798',1.0,1.0,56.0,32.4,2.0,4.0,0.0,43.75,1.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,b'9001104',2.0,2.0,72.0,30.7,1.0,0.0,5.0,37.50,4.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,b'9999365',1.0,2.0,56.0,30.3,2.0,4.0,6.0,18.75,1.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4792,b'9999510',1.0,2.0,50.0,28.9,1.0,3.0,0.0,68.75,1.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4793,b'9999862',2.0,2.0,61.0,23.7,1.0,0.0,0.0,93.75,4.0,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4794,b'9999865',2.0,2.0,61.0,25.0,2.0,0.0,0.0,100.00,4.0,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0


In [23]:
oai

,ID,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Pain,Followup_Pain,Baseline_Quality_of_life,Followup_Quality_of_life,PA_change,QOL_change,pain_change
0,b'9336746',75.0,1.0,27.2,1.0,3.0,1,4,0.0,0.0,75.00,100.00,3,25.00,0.0
1,b'9971275',77.0,1.0,33.6,1.0,3.0,1,4,50.0,30.0,37.50,50.00,3,12.50,-20.0
2,b'9223980',76.0,1.0,31.0,2.0,3.0,1,7,10.0,0.0,56.25,62.50,6,6.25,-10.0
3,b'9745368',69.0,1.0,31.8,2.0,1.0,1,3,30.0,30.0,62.50,68.75,2,6.25,0.0
4,b'9005132',74.0,1.0,29.6,1.0,1.0,1,5,70.0,40.0,50.00,43.75,4,-6.25,-30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,b'9417544',46.0,2.0,26.9,2.0,1.0,10,10,60.0,50.0,56.25,68.75,0,12.50,-10.0
1152,b'9484333',54.0,1.0,27.6,2.0,1.0,10,10,20.0,30.0,68.75,68.75,0,0.00,10.0
1153,b'9686777',56.0,2.0,27.5,1.0,1.0,10,6,40.0,50.0,43.75,56.25,-4,12.50,10.0
1154,b'9929102',57.0,1.0,33.6,2.0,1.0,10,10,40.0,90.0,68.75,68.75,0,0.00,50.0


In [24]:
oai_variables = ['V00CHIRCUR','V00HOMECV', 'V00NSAIDS','V00NSAIDRX', 'V10CHIRCUR', 'V10HOMECV', 'V10NSAIDS', 'V10NSAIDRX',
                 'P02KSURG','P01KSURGR','P01KRSR','P01ARTR','P01OTSURGR','P01KSURGL','P01ARTL','V01KSRGR12','V01SREPRKR','V01KRSR12','V01ARTR12','V01KSRGL12','V01KRSL12',
                 
                 'V00EKRSL','V01EKRSL','V00EKRSR','V01EKRSR'
                 
                 ]

In [25]:
# Function to clean byte-encoded IDs and convert them to strings
def clean_id_column(df, column_name):
    df[column_name] = df[column_name].astype(str)  # Convert to string
    df[column_name] = df[column_name].str.replace("b'", "").str.replace("'", "")  # Remove extra encoding
    return df

# Clean 'ID' in both dataframes
df = clean_id_column(df, 'ID')
oai = clean_id_column(oai, 'ID')

# Select only the needed columns from df
df_selected = df[['ID', 'V00CHIRCUR','V00HOMECV', 'V00NSAIDS','V00NSAIDRX', 'V10CHIRCUR', 'V10HOMECV', 'V10NSAIDS', 'V10NSAIDRX',
                 'P02KSURG','P01KSURGR','P01KRSR','P01ARTR','P01OTSURGR','P01KSURGL','P01ARTL','V01KSRGR12','V01SREPRKR','V01KRSR12','V01ARTR12','V01KSRGL12','V01KRSL12',
                 
                 'V00EKRSL','V01EKRSL','V00EKRSR','V01EKRSR'
                 ]]

# Merge to create merged_dfoai, keeping all rows from oai
merged_dfoai = pd.merge(oai, df_selected, how='left', on='ID')

In [26]:
print(merged_dfoai.shape)  # Expected: (1156, original oai columns + 4 new columns)

(1156, 40)


In [27]:
merged_dfoai

,ID,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Pain,Followup_Pain,...,V01KSRGR12,V01SREPRKR,V01KRSR12,V01ARTR12,V01KSRGL12,V01KRSL12,V00EKRSL,V01EKRSL,V00EKRSR,V01EKRSR
0,9336746,75.0,1.0,27.2,1.0,3.0,1,4,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1,9971275,77.0,1.0,33.6,1.0,3.0,1,4,50.0,30.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
2,9223980,76.0,1.0,31.0,2.0,3.0,1,7,10.0,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
3,9745368,69.0,1.0,31.8,2.0,1.0,1,3,30.0,30.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
4,9005132,74.0,1.0,29.6,1.0,1.0,1,5,70.0,40.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,9417544,46.0,2.0,26.9,2.0,1.0,10,10,60.0,50.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1152,9484333,54.0,1.0,27.6,2.0,1.0,10,10,20.0,30.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1153,9686777,56.0,2.0,27.5,1.0,1.0,10,6,40.0,50.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1154,9929102,57.0,1.0,33.6,2.0,1.0,10,10,40.0,90.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN


In [28]:
merged_dfoai.isna().sum()

ID                               0
Age                              0
Gender                           0
BMI                              0
Depression                       0
Employment_status                0
Baseline_Physical_Activity       0
Followup_Physical_Activity       0
Baseline_Pain                    0
Followup_Pain                    0
Baseline_Quality_of_life         0
Followup_Quality_of_life         0
PA_change                        0
QOL_change                       0
pain_change                      0
V00CHIRCUR                    1045
V00HOMECV                        2
V00NSAIDS                        2
V00NSAIDRX                       2
V10CHIRCUR                    1064
V10HOMECV                      306
V10NSAIDS                      255
V10NSAIDRX                     255
P02KSURG                         1
P01KSURGR                        1
P01KRSR                        891
P01ARTR                        894
P01OTSURGR                     891
P01KSURGL           

##### Create pain medication (nonprescription & prescription)

In [29]:
# V00NSAIDS   Baseline nonprescription
# V00NSAIDRX  Baseline prescription

# V10NSAIDS   Followup nonprescription
# V10NSAIDRX  Followup prescription

In [30]:
#baseline

merged_dfoai[['V00NSAIDS', 'V00NSAIDRX']] = merged_dfoai[['V00NSAIDS', 'V00NSAIDRX']].apply(pd.to_numeric, errors='coerce')
# Merge 
merged_dfoai['pain_medication_baseline'] = merged_dfoai[['V00NSAIDS', 'V00NSAIDRX']].max(axis=1)

print(merged_dfoai.pain_medication_baseline.value_counts(dropna=False))

pain_medication_baseline
0.0    786
1.0    368
NaN      2
Name: count, dtype: int64


In [31]:
#followup

merged_dfoai[['V10NSAIDS', 'V10NSAIDRX']] = merged_dfoai[['V10NSAIDS', 'V10NSAIDRX']].apply(pd.to_numeric, errors='coerce')
# Merge 
merged_dfoai['pain_medication_followup'] = merged_dfoai[['V10NSAIDS', 'V10NSAIDRX']].max(axis=1)

print(merged_dfoai.pain_medication_followup.value_counts(dropna=False))

pain_medication_followup
0.0    651
NaN    255
1.0    250
Name: count, dtype: int64


##### Create knee surgery replacement:

In [32]:
# Baseline
# Ensure values are treated as numeric (convert "NaN" strings to actual NaNs)
merged_dfoai[['V00EKRSL', 'V00EKRSR']] = merged_dfoai[['V00EKRSL', 'V00EKRSR']].apply(pd.to_numeric, errors='coerce')

# Create new column 'knee_replacement'
merged_dfoai['knee_replacement_baseline'] = merged_dfoai[['V00EKRSL', 'V00EKRSR']].max(axis=1)
print(merged_dfoai.knee_replacement_baseline.value_counts(dropna=False))

knee_replacement_baseline
0.0    1042
NaN      98
1.0      16
Name: count, dtype: int64


In [33]:
# Followup
# Ensure values are treated as numeric (convert "NaN" strings to actual NaNs)
merged_dfoai[['V01EKRSL', 'V01EKRSR']] = merged_dfoai[['V01EKRSL', 'V01EKRSR']].apply(pd.to_numeric, errors='coerce')

# Create new column 'knee_replacement'
merged_dfoai['knee_replacement_followup'] = merged_dfoai[['V01EKRSL', 'V01EKRSR']].max(axis=1)
print(merged_dfoai.knee_replacement_followup.value_counts(dropna=False))

knee_replacement_followup
NaN    1053
0.0     100
1.0       3
Name: count, dtype: int64


##### Value counts

In [34]:
oai_variables = [
"V00CHIRCUR"
,"V10CHIRCUR"
,"V00HOMECV"
,"V10HOMECV"
,"V00NSAIDS"
,"V10NSAIDS"
,"V00NSAIDRX"
,"V10NSAIDRX"
,"P02KSURG"
,"P01KSURGR"
,"P01KRSR"
,"P01ARTR"
,"P01OTSURGR"
,"P01KSURGL"
,"P01ARTL"
,"V01KSRGR12"
,"V01SREPRKR"
,"V01KRSR12"
,"V01ARTR12"
,"V01KSRGL12"
,"V01KRSL12",
'V00EKRSL','V01EKRSL','V00EKRSR','V01EKRSR'

                 ]

# Display value counts including NaN for each variable
for var in oai_variables:
    print(f"\nValue counts for {var}:\n")
    print(merged_dfoai[var].value_counts(dropna=False))



Value counts for V00CHIRCUR:

V00CHIRCUR
NaN    1045
1.0      65
0.0      46
Name: count, dtype: int64

Value counts for V10CHIRCUR:

V10CHIRCUR
NaN    1064
1.0      60
0.0      32
Name: count, dtype: int64

Value counts for V00HOMECV:

V00HOMECV
0.0    1147
1.0       7
NaN       2
Name: count, dtype: int64

Value counts for V10HOMECV:

V10HOMECV
0.0    847
NaN    306
1.0      3
Name: count, dtype: int64

Value counts for V00NSAIDS:

V00NSAIDS
0.0    857
1.0    297
NaN      2
Name: count, dtype: int64

Value counts for V10NSAIDS:

V10NSAIDS
0.0    739
NaN    255
1.0    162
Name: count, dtype: int64

Value counts for V00NSAIDRX:

V00NSAIDRX
0.0    1054
1.0     100
NaN       2
Name: count, dtype: int64

Value counts for V10NSAIDRX:

V10NSAIDRX
0.0    797
NaN    255
1.0    104
Name: count, dtype: int64

Value counts for P02KSURG:

P02KSURG
0.0    746
1.0    409
NaN      1
Name: count, dtype: int64

Value counts for P01KSURGR:

P01KSURGR
0.0    890
1.0    265
NaN      1
Name: count, dtype